In [10]:
import os
import torch

from pytorch_lightning import Trainer, seed_everything

from bci_aic3.config import load_model_config, load_training_config
from bci_aic3.paths import (
    PROCESSED_DATA_DIR,
    RAW_DATA_DIR,
    MI_CONFIG_PATH,
    SSVEP_CONFIG_PATH,
)
from bci_aic3.train import BCILightningModule, setup_callbacks, create_processed_data_loaders
from bci_aic3.util import rec_cpu_count


In [11]:
# Code necessary to create reproducible runs
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
seed_everything(42, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)


Seed set to 42


In [ ]:
from pathlib import Path
from typing import Tuple


def train_model(config_path: Path, verbose: bool = True) -> Tuple[Trainer, BCILightningModule]:
    model_config = load_model_config(config_path)
    training_config = load_training_config(config_path)

    max_num_workers = rec_cpu_count()

    # Create data loaders
    train_loader, val_loader = create_processed_data_loaders(
        processed_data_dir=PROCESSED_DATA_DIR,
        task_type=model_config.task_type,
        batch_size=training_config.batch_size,
        num_workers=max_num_workers,
    )
    
    # Create Lightning module
    model = BCILightningModule(
        num_classes=model_config.num_classes,
        num_channels=model_config.num_channels,
        sequence_length=model_config.new_sequence_length,
        lr=training_config.learning_rate,
    )

    # Setup callbacks
    callbacks = setup_callbacks(model_config, verbose=verbose)
    
    # Create trainer
    trainer = Trainer(
        max_epochs=training_config.epochs,
        callbacks=callbacks,
        accelerator="auto",  # Automatically uses GPU if available
        devices="auto",  # Uses all available devices
        deterministic=True,  # For reproducibility
        log_every_n_steps=10,
    )
    
    trainer.fit(model, train_loader, val_loader)
    
    return trainer, model

In [ ]:
trainer, model = train_model(config_path=SSVEP_CONFIG_PATH)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
p:\Programming\AIC3\repo\bci_aic3\.venv\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory P:\Programming\AIC3\repo\bci_aic3\checkpoints\SSVEP exists and is not empty.

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | EEGNet             | 36.2 K | train
1 | criterion      | CrossEntropyLoss   | 0      | train
2 | train_accuracy | MulticlassAccuracy | 0      | train
3 | val_accuracy   | MulticlassAccuracy | 0      | train
4 | train_f1       | MulticlassF1Score  | 0      | train
5 | val_f1         | MulticlassF1Score  | 0      | train
--------------------------------------------------------------
36.2 K    Trainable params
0         Non-trainable params
36.2 K    Total params
0.145     Total estimated model params size (MB)
20        Modules in tra

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

p:\Programming\AIC3\repo\bci_aic3\.venv\Lib\site-packages\torch\nn\modules\conv.py:549: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Convolution.cpp:1037.)
  return F.conv2d(


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [9]:
from bci_aic3.preprocess import load_and_preprocess_for_inference
from bci_aic3.paths import RAW_DATA_DIR
from bci_aic3.inference import create_inference_data_loader


test_dataset = load_and_preprocess_for_inference(csv_file="test.csv",
                                        base_path=RAW_DATA_DIR,
                                        task_type="SSVEP")
test_loader = create_inference_data_loader(test_dataset=test_dataset)

# model.eval()

# with torch.no_grad():
#     data_batch = next(iter(test_loader))

#     preds = model(data_batch[0])
    
def predict_batch(model, data_loader, device):
    """Make predictions on a batch of data."""
    model.eval()
    predictions = []

    with torch.no_grad():
        for x_batch, _ in data_loader:
            x_batch = x_batch.to(device)
            outputs = model(x_batch)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            predictions.extend(preds.tolist())

    return predictions

predict_batch(model=model,
              data_loader=test_loader, 
              device="cuda" if torch.cuda.is_available() else "cpu")

ImportError: cannot import name 'load_and_preprocess_for_inference' from 'bci_aic3.preprocess' (P:\Programming\AIC3\repo\bci_aic3\src\bci_aic3\preprocess.py)

In [70]:
preds.argmax(dim=1)

tensor([1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 1])

In [71]:
# from bci_aic3.paths import REVERSE_LABEL_MAPPING_PATH
from bci_aic3.paths import CONFIG_DIR


REVERSE_LABEL_MAPPING_PATH = CONFIG_DIR / "reverse_label_mapping.json"


reverse_label_mapping = read_json_to_dict(REVERSE_LABEL_MAPPING_PATH)
preds_labels = [reverse_label_mapping[str(p.item())] for p in preds.argmax(dim=1)]
preds_labels

['Right',
 'Right',
 'Right',
 'Left',
 'Right',
 'Right',
 'Right',
 'Right',
 'Left',
 'Left',
 'Right',
 'Right',
 'Left',
 'Left',
 'Left',
 'Left',
 'Right',
 'Right',
 'Right',
 'Right',
 'Right',
 'Left',
 'Left',
 'Left',
 'Right',
 'Right',
 'Right',
 'Right',
 'Right',
 'Right',
 'Left',
 'Left',
 'Left',
 'Left',
 'Left',
 'Left',
 'Left',
 'Left',
 'Left',
 'Left',
 'Left',
 'Left',
 'Right',
 'Left',
 'Left',
 'Right',
 'Left',
 'Left',
 'Right',
 'Right']

In [72]:
reverse_label_mapping

{'0': 'Left', '1': 'Right', '2': 'Forward', '3': 'Backward'}

In [73]:
from bci_aic3.paths import RUNS_DIR


scripted = model.to_torchscript()

torch.jit.save(scripted, RUNS_DIR / "mi_scripted.pt")

RuntimeError: 
'Tensor' object has no attribute or method '__init__'.:
  File "P:\Programming\AIC3\repo\bci_aic3\src\bci_aic3\train.py", line 43
        # scheduler_name: str = None,
    ):
        super().__init__()
        ~~~~~~~~~~~~~~ <--- HERE
        self.save_hyperparameters()  # Automatically saves all __init__ params
    
